In [1]:
import yfinance as yf
import pandas as pd
from zipline.api import order_target_percent, symbol, set_commission, set_benchmark, order,record, order_target, get_open_orders
from zipline.utils.calendar_utils import get_calendar
from zipline import run_algorithm
import matplotlib.pyplot as plt
import numpy as np

In [21]:
start_date = '2010-01-01'
end_date = '2023-01-01'

tesla_data = yf.download("TSLA", start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [22]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current Working Directory:", current_directory)


Current Working Directory: /home/subrina/Trading with python/Zipline/custom_data


In [23]:
tesla_data.to_csv('./tesla_data.csv')

In [24]:
tesla_data = pd.read_csv('./tesla_data.csv')

In [25]:
tesla_data = tesla_data.drop('Adj Close', axis= 1)
tesla_data.head()

,Date,Open,High,Low,Close,Volume
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500


In [26]:
new_column_names = {'Date': 'date', 'Open':'open', 'High': 'high', 'Low':'low', 'Close': 'close', 'Volume': 'volume'}

tesla_data.rename(columns=new_column_names, inplace=True)
tesla_data.head()

,date,open,high,low,close,volume
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,281494500
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,257806500
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,123282000
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,77097000
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,103003500


In [27]:
column_data_types = tesla_data.dtypes
column_data_types

date       object
open      float64
high      float64
low       float64
close     float64
volume      int64
dtype: object

In [28]:
tesla_data.iloc[:, 1:] = tesla_data.iloc[:, 1:].apply(lambda x: x.astype(float))
column_data_types = tesla_data.dtypes
column_data_types

date       object
open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object

In [29]:
tesla_data['timestamp'] = pd.to_datetime(tesla_data['date'])
tesla_data = tesla_data.reindex(columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
tesla_data['timestamp'] = tesla_data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
tesla_data.head()

,timestamp,open,high,low,close,volume
0,2010-06-29 00:00:00.000000,1.266667,1.666667,1.169333,1.592667,281494500.0
1,2010-06-30 00:00:00.000000,1.719333,2.028000,1.553333,1.588667,257806500.0
2,2010-07-01 00:00:00.000000,1.666667,1.728000,1.351333,1.464000,123282000.0
3,2010-07-02 00:00:00.000000,1.533333,1.540000,1.247333,1.280000,77097000.0
4,2010-07-06 00:00:00.000000,1.333333,1.333333,1.055333,1.074000,103003500.0


In [30]:
tesla_data = tesla_data.sort_values(by='timestamp', ascending=True)
tesla_data = tesla_data.reset_index(drop=True)
tesla_data

,timestamp,open,high,low,close,volume
0,2010-06-29 00:00:00.000000,1.266667,1.666667,1.169333,1.592667,281494500.0
1,2010-06-30 00:00:00.000000,1.719333,2.028000,1.553333,1.588667,257806500.0
2,2010-07-01 00:00:00.000000,1.666667,1.728000,1.351333,1.464000,123282000.0
3,2010-07-02 00:00:00.000000,1.533333,1.540000,1.247333,1.280000,77097000.0
4,2010-07-06 00:00:00.000000,1.333333,1.333333,1.055333,1.074000,103003500.0
...,...,...,...,...,...,...
3145,2022-12-23 00:00:00.000000,126.370003,128.619995,121.019997,123.150002,166989700.0
3146,2022-12-27 00:00:00.000000,117.500000,119.669998,108.760002,109.099998,208643400.0
3147,2022-12-28 00:00:00.000000,110.349998,116.269997,108.239998,112.709999,221070500.0
3148,2022-12-29 00:00:00.000000,120.389999,123.570000,117.500000,121.820000,221923300.0


In [13]:
# tesla_data = tesla_data.groupby('timestamp').last().reset_index()
# tesla_data

,timestamp,open,high,low,close,volume
0,2010-06-29 00:00:00.000000,1.266667,1.666667,1.169333,1.592667,281494500.0
1,2010-06-30 00:00:00.000000,1.719333,2.028000,1.553333,1.588667,257806500.0
2,2010-07-01 00:00:00.000000,1.666667,1.728000,1.351333,1.464000,123282000.0
3,2010-07-02 00:00:00.000000,1.533333,1.540000,1.247333,1.280000,77097000.0
4,2010-07-06 00:00:00.000000,1.333333,1.333333,1.055333,1.074000,103003500.0
...,...,...,...,...,...,...
3145,2022-12-23 00:00:00.000000,126.370003,128.619995,121.019997,123.150002,166989700.0
3146,2022-12-27 00:00:00.000000,117.500000,119.669998,108.760002,109.099998,208643400.0
3147,2022-12-28 00:00:00.000000,110.349998,116.269997,108.239998,112.709999,221070500.0
3148,2022-12-29 00:00:00.000000,120.389999,123.570000,117.500000,121.820000,221923300.0


In [31]:
tesla_data.columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [32]:
tesla_data.set_index('timestamp', inplace=True)
tesla_data.to_csv('./daily/tesla_data.csv')

In [38]:
!zipline bundles

csvdir <no ingestions>
quandl 2023-12-11 18:58:51.384085
quandl 2023-12-11 18:58:18.790981
quandl 2023-12-11 17:27:05.201175
quandl 2023-12-11 17:16:04.140642
quandl 2023-12-11 09:19:57.807125
quandl 2023-12-11 09:18:50.209032
quandl 2023-12-11 09:14:10.865540
quandl 2023-12-11 06:27:30.829838
quandl 2023-12-10 12:31:04.001536
quandl 2023-12-10 12:22:46.640028
quantopian-quandl <no ingestions>


In [42]:
!zipline ingest -b tesla_bundle

[2023-12-12T16:52:51+0600-INFO][zipline.data.bundles.core]
 Ingesting tesla_bundle
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [####################################]   
[2023-12-12T16:52:51+0600-INFO][root]
 creating /home/subrina/.zipline/data/tesla_bundle/2023-12-12T10;52;50.550990/daily_equities.bcolz
[2023-12-12T16:52:51+0600-INFO][root]
 copying /tmp/tmpdco7tyf_/tesla_bundle/2023-12-12T10;52;50.550990/daily_equities.bcolz/__attrs__ -> /home/subrina/.zipline/data/tesla_bundle/2023-12-12T10;52;50.550990/daily_equities.bcolz
[2023-12-12T16:52:51+0600-INFO][root]
 creating /home/subrina/.zipline/data/tesla_bundle/2023-12-12T10;52;50.550990/daily_equities.bcolz/close
[2023-12-12T16:52:51+0600-INFO][root]
 copying /tmp/tmpdco7tyf_/tesla_bundle/2023-12-12T10;52;50.550990/daily_equities.bcolz/close/__attrs__ -> /home/subrina/.zipline/data/tesla_bundle/2023-12-12T10;52;50.550990/daily_equities.bcolz/close
[2023-12-12T16:52:51+060

In [41]:
from zipline.data import bundles

bundles.ingest('tesla_bundle')


UnknownBundle: No bundle registered with the name 'tesla_bundle'